# Project Luther

The goal of project luther is to create a linear regression model for supervised machine learning. We also need to incorporate web scrapping into the project to get practice finding what we need.  

I will be pulling data from Ted.com.  Specifically, I will be grabbing view count, tags and the published date.  We also will initiate our proxies here, and I will tell why we need them when they are employed below.

Packages needed for this portion:
 * Pandas
 * Numpy
 * BeautifulSoup
 * Requests
 * Regex

In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
import re
import time
import csv

In [61]:
#Proxies need to be changed each run until data is collected!
proxies = {
    "http": 'http://108.61.87.166:8080', 
    "https": 'http://108.61.87.166:8080'
}
prox2 = {
    "http": 'http://144.202.108.231:8888', 
    "https": 'http://144.202.108.231:8888'
}
prox3 = {
    "http": 'http://52.229.52.86:3128', 
    "https": 'http://52.229.52.86:3128'
}
prox4 = {
    "http": 'http://35.169.210.105:3389', 
    "https": 'http://35.169.210.105:3389'
}
base = 'https://www.ted.com'
start_url = 'https://www.ted.com/talks'

response = rq.get(start_url)

In [38]:
page = response.text
mainpage = soup(page,"lxml")

In [39]:
talks = re.compile('/talks/')
views = re.compile("Related Tags")

In [40]:
def GetVid(mainlist):
    vid_str = []
    l = len(mainlist)-1
    for i in range(0,l,2):
        new_str = str(mainlist[i])
        vid_str.append(new_str.split('"')[5])
    return vid_str

In [41]:
#Getting all cases where we have a /talks url
list_1 = mainpage.find_all(href=talks)


In [42]:
#Pulling a list of all video end url links
lnk_list = GetVid(list_1)
lnk_list

['/talks/tracee_ellis_ross_a_woman_s_fury_holds_lifetimes_of_wisdom',
 '/talks/dayo_ogunyemi_visions_of_africa_s_future_from_african_filmmakers',
 '/talks/clemantine_wamariya_war_and_what_comes_after',
 '/talks/gwynne_shotwell_spacex_s_plan_to_fly_you_across_the_globe_in_30_minutes',
 '/talks/diane_wolk_rogers_a_parkland_teacher_s_homework_for_us_all',
 '/talks/zachary_r_wood_why_it_s_worth_listening_to_people_we_disagree_with',
 '/talks/nancy_rabalais_the_dead_zone_of_the_gulf_of_mexico',
 '/talks/mark_tyndall_the_harm_reduction_model_of_drug_addiction_treatment',
 '/talks/hannah_burckstummer_a_printable_flexible_organic_solar_cell',
 '/talks/yasin_kakande_what_s_missing_in_the_global_debate_over_refugees',
 '/talks/robin_steinberg_what_if_we_ended_the_injustice_of_bail',
 '/talks/jaron_lanier_how_we_need_to_remake_the_internet',
 '/talks/malika_whitley_how_the_arts_help_homeless_youth_heal_and_build',
 '/talks/lera_boroditsky_how_language_shapes_the_way_we_think',
 '/talks/jose_andre

## Creating a list for all ted talks

The ted talks pagination has the url /talks?page=3, therefore we can simplify this to a 'talks?page=' + a list of numbers from 2 to 77 in '77'.  We need to cycle through group of proxies to avoid getting blocked from the site.  We never get permanently 'blocked', but we can get blocked temporarily.

In [43]:
page_lst = [str(i) for i in range(2,78)]
mid = '/talks?page='

In [44]:
url_list = [base+mid+i for i in page_lst]
url_list.insert(0,'https://www.ted.com/talks')

In [45]:
def GetVidLst(vurl, prox = None):
    resp = rq.get(vurl, prox)
    ppage = resp.text
    vidpage = soup(ppage,"lxml")
    vlist = vidpage.find_all(href=talks)
    vid_lnk_list = GetVid(vlist)
    return vid_lnk_list

In [46]:
#Creating a list of all the ted vides
fin = []
count=0
proxy = None
for i in url_list:
    count+=1
    fin +=GetVidLst(i, proxy)
    time.sleep(5)
    if count == 30 or count == 60 or count == 90 or count == 120:
        time.sleep(10)
        if count ==30:proxy = proxies
        if count ==60: proxy = prox2
        if count ==90: proxy = prox3
        if count ==120: 
            proxy = None 
            count = 10

In [62]:
fin[0]

'/talks/tracee_ellis_ross_a_woman_s_fury_holds_lifetimes_of_wisdom'

## Create Ted Talk Dictionary

Now that we have our list of all the videos, we are going to create a dictionary with the following format:
{'Title':\['Views', \['Tags'\], Date Posted\]}.  

To accomplish this, I first did all the work on a single page, then I will create functions for each step and call them in a for loop over the list 'fin'.

In [79]:
#Changes views string to an integer
def view_to_int(viewstring):
    viewstring= viewstring.replace('\n', '').replace(',', '')
    return int(viewstring)

In [80]:
def get_Views(site_text):
    views_list = []
    viewint=0
    views = site_text.find_all(class_=" d:n f-w:700 f:.9 f:1@xxl c:white ")
    str_list2 = str(views).split('<span')[1].split('\t')
    viewint = view_to_int(str_list2[5])
    return viewint

In [81]:
def get_ratings(vidpage):
    super_comment = vidpage.find_all(text = re.compile('ratings'))
    the_max = max(super_comment, key=len)
    super_comment=the_max.split('ratings')
    super_comment = super_comment[1].split("\"name\":")
    super_comment
    target_ratings = ['Confusing', 'Funny',  'Unconvincing', 'Longwinded', 'Inspiring', 'Fascinating', 'Jaw-dropping']
    count = 0
    rate_dict = {}
    for i in super_comment:
        if count in [2,5,8,9,10,11,12]:
            str_list = i.split(",")
            rate_name = str_list[0].replace("\"",'')
            rate_count = str_list[1].replace('\"count\":','').split('}')[0]
            rate_dict[rate_name] = float(rate_count)
        count+=1
    return rate_dict

In [82]:
#Cleaning up site text to get tags
def Cleanup(site_text):
    super_comment = site_text.find_all(text = re.compile('tag'))
    the_max = max(super_comment, key=len)
    super_comment=the_max.split('tags')
    super_comment = super_comment[1].split("\"")
    tag=[]
    tag =[super_comment[i] for i in range(2,9,2)]
    return tag

In [83]:
#Cleaning up site text to get tags
def Cleanup(site_text):
    super_comment = site_text.find_all(text = re.compile("\"ratings\""))
    the_max = max(super_comment, key=len)
    super_comment=the_max.split("\"ratings\"")
    super_comment = super_comment[1].split("\"")
    tag=[]
    tag =[super_comment[i] for i in range(2,9,2)]
    return tag

In [84]:
def getDate(site_text):
    super_comment = str(site_text.find_all(text = re.compile('recorded_at')))
    inter = re.findall(re.compile('"recorded_at"'), super_comment)
    intermed = super_comment.split(inter[0])
    intermed = intermed[1].split(':')
    date = intermed[1].replace('T00', '').replace('"', "")
    return date

In [85]:
#Creating a list of all the ted vides
data = {}
count=0
tot_views=[]
proxy = prox4
for i in fin:
    time.sleep(15)
    count+=1
    curl = base+i
    response = rq.get(curl, proxy)
    web_page = response.text
    webpage = soup(web_page,"lxml")
    date = getDate(webpage)
    tag_list = Cleanup(webpage)
    tot_views = get_Views(webpage)
    ratings = get_ratings(webpage)
    data[i] = [date,tot_views,tag_list,ratings]
    if count == 20 or count == 40 or count == 60 or count == 80:
        time.sleep(10)
        if count ==20:proxy = proxies
        if count ==40: proxy = prox2
        if count ==60: proxy = prox4
        if count ==80: 
            proxy = prox3
            count = 0



ValueError: could not convert string to float: '"viewed_count":772886'

In [86]:
len(data)

258

In [87]:
#Writing a pickle file
import pickle

with open('my_data.pkl', 'wb') as picklefile:
    pickle.dump(data, picklefile)

In [464]:
#reading a pickle file
with open("my_data.pkl", 'rb') as picklefile: 
    my_old_data = pickle.load(picklefile)

## To Pandas

Now that we have our dictionary of values, we are changing it to a pandas dataframe for easier manipulation.

There will then be some prelimnary exploring to see if it is formatted correctly or if more manipulation is needed.

In [465]:
my_old_data

{'/talks/adam_alter_why_our_screens_make_us_less_happy': ['2017-04-24',
  2915234,
  ['Addiction', 'computers', 'interface design', 'psychology']],
 '/talks/adong_judith_how_i_use_art_to_bridge_misunderstanding': ['2017-08-27',
  540027,
  ['Africa', 'TED Fellows', 'activism', 'art']],
 '/talks/alastair_gray_how_fake_handbags_fund_terrorism_and_organized_crime': ['2017-11-14',
  1127728,
  ['business', 'corruption', 'crime', 'terrorism']],
 '/talks/alexander_wagner_what_really_motivates_people_to_be_honest_in_business': ['2016-11-11',
  1336721,
  ['TEDx', 'business', 'corruption', 'economics']],
 '/talks/alexis_charpentier_how_record_collectors_find_lost_music_and_preserve_our_cultural_heritage': ['2016-11-12',
  752230,
  ['TEDx', 'art', 'culture', 'music']],
 '/talks/alvin_irby_how_to_inspire_every_child_to_be_a_lifelong_reader': ['2017-11-28',
  774478,
  ['TED Residency', 'children', 'education', 'identity']],
 '/talks/amar_inamdar_the_thrilling_potential_for_off_grid_solar_energy

In [466]:
# if I want the key's to be rows, pass orient = 'index'
data_dict = pd.from_dict(my_old_data)

AttributeError: module 'pandas' has no attribute 'from_dict'

In [ ]:
data_dict.describe()

In [ ]:
data_dict.head()

## More Data

There is data from a kaggle that incorporates similar things above, and I will use it to enrich my data.  I will also substitute using it now because I am still troubleshooting pulling data from ted.com

In [69]:
import pickle
#reading a pickle file
with open("url_list.pkl", 'rb') as picklef: 
    kag_url = pickle.load(picklef)

In [70]:
kag_url

2288    https://www.ted.com/talks/deeyah_khan_what_we_...
2289    https://www.ted.com/talks/deepika_kurup_a_youn...
2290    https://www.ted.com/talks/sarah_parcak_help_di...
2291    https://www.ted.com/talks/alan_smith_why_we_re...
2292    https://www.ted.com/talks/eduardo_briceno_how_...
2293    https://www.ted.com/talks/sue_klebold_my_son_w...
2294    https://www.ted.com/talks/nagin_cox_what_time_...
2295    https://www.ted.com/talks/maurice_conti_the_in...
2296    https://www.ted.com/talks/thordis_elva_tom_str...
2297    https://www.ted.com/talks/joshua_smith_new_nan...
2298    https://www.ted.com/talks/jeff_speck_4_ways_to...
2299    https://www.ted.com/talks/aala_el_khani_what_i...
2300    https://www.ted.com/talks/miriam_zoila_perez_h...
2301    https://www.ted.com/talks/rodrigo_y_gabriela_a...
2302    https://www.ted.com/talks/amy_adele_hasinoff_h...
2303    https://www.ted.com/talks/lara_setrakian_3_way...
2304    https://www.ted.com/talks/salil_dudani_how_jai...
2305    https:

In [71]:
#Creating a list of all the ted vides
data = {}
count=0
tot_views=[]
proxy = proxies
for i in kag_url:
    time.sleep(30)
    count+=1
    response = rq.get(i, proxy)
    web_page = response.text
    webpage = soup(web_page,"lxml")
    date = getDate(webpage)
    tag_list = Cleanup(webpage)
    tot_views = get_Views(webpage)
    ratings = get_ratings(webpage)
    data[i] = [date,tot_views,tag_list,ratings]
    if count == 20 or count == 40 or count == 60 or count == 80:
        time.sleep(10)
        if count ==20:proxy = proxies
        if count ==40: proxy = prox2
        if count ==60: proxy = prox4
        if count ==80: 
            proxy = prox3
            count = 0

ValueError: could not convert string to float: '"low":"https://download.ted.com/talks/LuxNarayan_2017S-low-bg.mp4"'

In [90]:
with open('test_data.pkl', 'wb') as picklefile:
    pickle.dump(data, picklefile)

In [91]:
len(data)
data

{'/talks/adong_judith_how_i_use_art_to_bridge_misunderstanding': ['2017-08-27',
  573793,
  [':8,', ':', ',', ':19},{'],
  {'Beautiful': 12.0,
   'Confusing': 0.0,
   'Funny': 1.0,
   'Ingenious': 3.0,
   'Inspiring': 24.0,
   'Jaw-dropping': 1.0,
   'Longwinded': 0.0}],
 '/talks/alastair_gray_how_fake_handbags_fund_terrorism_and_organized_crime': ['2017-11-14',
  1139540,
  [':8,', ':', ',', ':231},{'],
  {'Beautiful': 9.0,
   'Fascinating': 31.0,
   'Inspiring': 25.0,
   'Jaw-dropping': 34.0,
   'Obnoxious': 8.0,
   'Persuasive': 83.0,
   'Unconvincing': 10.0}],
 '/talks/alexander_wagner_what_really_motivates_people_to_be_honest_in_business': ['2016-11-11',
  1342139,
  [':10,', ':', ',', ':137},{'],
  {'Fascinating': 97.0,
   'Funny': 17.0,
   'Informative': 149.0,
   'Ingenious': 22.0,
   'Jaw-dropping': 4.0,
   'Obnoxious': 6.0,
   'Unconvincing': 5.0}],
 '/talks/alexis_charpentier_how_record_collectors_find_lost_music_and_preserve_our_cultural_heritage': ['2016-11-12',
  765308,


In [78]:
webpage

<!DOCTYPE html>
<!--[if lt IE 8]> <html class="no-js loggedout oldie ie7" lang="en"> <![endif]--><!--[if IE 8]> <html class="no-js loggedout oldie ie8" lang="en"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js loggedout" lang="en"><!--<![endif]-->
<head>
<script>
  (function (H){
  H.className=H.className.replace(/\bno-js\b/,'js');
  if (('; '+document.cookie).match(/; _ted_user_id=/)) H.className=H.className.replace(/\bloggedout\b/,'loggedin');
  })(document.documentElement)
</script><meta charset="utf-8"/>
<title>Lux Narayan: What I learned from 2,000 obituaries | TED Talk</title>
<link href="https://www.ted.com/talks/lux_narayan_what_i_learned_from_2_000_obituaries" rel="canonical"/>
<meta content="app-id=376183339,app-argument=https://www.ted.com/talks/lux_narayan_what_i_learned_from_2_000_obituaries?utm_source=ted.com&amp;utm_medium=referral&amp;utm_campaign=smart_app_banner" name="apple-itunes-app"/>
<link href="https://www.ted.com/talks/lux_narayan_what_i_learned_from_2_00